In [ ]:
NLP Practical 3

In [1]:
import numpy as np
import os
from random import shuffle
import re
import sklearn as sk
import matplotlib.pyplot as plt
import random

In [2]:
import urllib.request
import zipfile
import lxml.etree

In [3]:
import tensorflow as tf

In [4]:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
    
raw_text = doc.xpath('//content/text()')
raw_label = doc.xpath('//head/keywords/text()')

del doc

In [8]:
talk_sentences = []
talknum = len(raw_text)

for i in range(talknum):
    temp = re.sub(r'\([^)]*\)', '', raw_text[i])
    temp = re.sub(r'\n', '', raw_text[i])
    temp = temp.split('.')
    talk_sentences.append(temp)

In [9]:
print(talk_sentences[0][1])

To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation


In [10]:
talk_sentence_word = []

for talk in talk_sentences:
    temp = []
    for sent in talk:
        tokens = re.sub(r"[^a-z0-9]+", " ", sent.lower()).split()
        temp.append(tokens)#
    talk_sentence_word.append(temp)

In [12]:
talk_sentence_word[0][1]

['to',
 'me',
 'the',
 'real',
 'real',
 'solution',
 'to',
 'quality',
 'growth',
 'is',
 'figuring',
 'out',
 'the',
 'balance',
 'between',
 'two',
 'activities',
 'exploration',
 'and',
 'exploitation']

In [13]:
freq = {}

for talk in talk_sentence_word:
    for sent in talk:
        for word in sent:
            if word in freq:
                freq[word] += 1
            else:
                freq[word] = 1

In [14]:
from collections import Counter

counts_ted_top1000 = []
words_top_ted = []

for word, count in Counter(freq).most_common(1000):
    counts_ted_top1000.append(count)
    words_top_ted.append(word)

In [15]:
talk_sentence_word_nostop = talk_sentence_word

for stop in words_top_ted[:200]:
    for talk in talk_sentence_word_nostop:
        for sent in talk:
            for word in sent:
                if word == stop:
                    sent.remove(word)

In [16]:
talk_sentence_word_nostop[0][1]

['real',
 'real',
 'solution',
 'quality',
 'growth',
 'figuring',
 'balance',
 'between',
 'activities',
 'exploration',
 'exploitation']

In [17]:
np.shape(talk_sentence_word_nostop)

(2085,)

In [18]:
input_label = []

for i in range(len(raw_label)):
    temp = re.sub(r' ', '', raw_label[i])
    input_label.append(temp.split(','))
    
#print(input_label_prepro)

In [19]:
labels_binary = []

for i in range(len(input_label)):
    temp = np.zeros(3)
    if 'technology' in '~'.join(input_label[i]):
        temp[0] = 1
    if 'entertainment' in '~'.join(input_label[i]):
        temp[1] = 1
    if 'design' in '~'.join(input_label[i]):
        temp[2] = 1
    labels_binary.append(temp)
        
print(input_label[9])
print(labels_binary[:10])

['talks', 'Senses', 'augmentedreality', 'brain', 'computers', 'creativity', 'cyborg', 'demo', 'design', 'engineering', 'entrepreneur', 'innovation', 'interfacedesign', 'invention', 'neuroscience', 'potential', 'prediction', 'productdesign', 'technology', 'visualizations']
[array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  1.]), array([ 1.,  0.,  1.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 1.,  0.,  1.])]


In [20]:
temp = list(zip(talk_sentence_word_nostop, labels_binary))
random.shuffle(temp)
talk_sentence_word_nostop_shuffle, labels_binary_shuffle = zip(*temp)

In [21]:
labels_onehot = []
for i in range(len(input_label)):
    temp = np.zeros(8)
    temp[labels_binary[i][0]*4+labels_binary[i][1]*2+labels_binary[i][2]*1] = 1
    labels_onehot.append(temp)

/home/scratch/ms16lg2/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [22]:
labels_onehot_shuffle = []
for i in range(len(input_label)):
    temp = np.zeros(8)
    temp[labels_binary_shuffle[i][0]*4+labels_binary_shuffle[i][1]*2+labels_binary_shuffle[i][2]*1] = 1
    labels_onehot_shuffle.append(temp)

/home/scratch/ms16lg2/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [23]:
train_data = talk_sentence_word_nostop[:1835]
test_data = talk_sentence_word_nostop[1835:]

train_labels = labels_binary[:1835]
test_labels = labels_binary[1835:]

train_labels_onehot = labels_onehot[:1835]
test_labels_onehot = labels_onehot[1835:]

In [24]:
train_data_shuffle = talk_sentence_word_nostop_shuffle[:1835]
test_data_shuffle = talk_sentence_word_nostop_shuffle[1835:]

train_labels_shuffle = labels_binary_shuffle[:1835]
test_labels_shuffle = labels_binary_shuffle[1835:]

train_labels_onehot_shuffle = labels_onehot_shuffle[:1835]
test_labels_onehot_shuffle = labels_onehot_shuffle[1835:]

In [27]:
print(np.shape(talk_sentence_word_nostop))
print(np.shape(labels_binary))
print(np.shape(labels_onehot))
print(np.shape(train_data))
print(np.shape(train_labels))
print(np.shape(train_labels_onehot))
print(np.shape(test_data))
print(np.shape(test_labels))
print(np.shape(test_labels_onehot))

(2085,)
(2085, 3)
(2085, 8)
(1835,)
(1835, 3)
(1835, 8)
(250,)
(250, 3)
(250, 8)


In [30]:
print(np.shape(talk_sentence_word_nostop_shuffle))
print(np.shape(labels_binary_shuffle))
print(np.shape(labels_onehot_shuffle))
print(np.shape(train_data_shuffle))
print(np.shape(train_labels_shuffle))
print(np.shape(train_labels_onehot_shuffle))
print(np.shape(test_data_shuffle))
print(np.shape(test_labels_shuffle))
print(np.shape(test_labels_onehot_shuffle))

(2085,)
(2085, 3)
(2085, 8)
(1835,)
(1835, 3)
(1835, 8)
(250,)
(250, 3)
(250, 8)


In [31]:
check = np.asarray(train_labels_onehot)
print(check.sum(0) / check.sum())
check = np.asarray(test_labels_onehot)
print(check.sum(0) / check.sum())

[ 0.57493188  0.0746594   0.0719346   0.00980926  0.17711172  0.06376022
  0.01525886  0.01253406]
[ 0.292  0.108  0.164  0.024  0.224  0.112  0.032  0.044]


In [32]:
check = np.asarray(train_labels_onehot_shuffle)
print(check.sum(0) / check.sum())
check = np.asarray(test_labels_onehot_shuffle)
print(check.sum(0) / check.sum())

[ 0.54277929  0.08119891  0.08392371  0.01144414  0.18583106  0.06376022
  0.01580381  0.01525886]
[ 0.528  0.06   0.076  0.012  0.16   0.112  0.028  0.024]


In [34]:
train_lenghts = []

for talk in train_data:
    N = 0
    for sent in talk:
        for word in sent:
            N += 1
    train_lenghts.append(N)

In [35]:
train_lenghts[:10]

[475, 807, 852, 493, 971, 823, 676, 692, 977, 622]

In [36]:
test_lenghts = []

for talk in test_data:
    N = 0
    for sent in talk:
        for word in sent:
            N += 1
    test_lenghts.append(N)

In [59]:
test_lenghts[:10]

[208, 1014, 826, 203, 1718, 241, 100, 979, 944, 1273]

In [43]:
train_talk_word = []

for talk in train_data:
    temp = []
    for sent in talk:
        for word in sent:
            temp.append(word)
    train_talk_word.append(temp)
    
print(len(train_talk_word[0]))

475


In [44]:
test_talk_word = []

for talk in test_data:
    temp = []
    for sent in talk:
        for word in sent:
            temp.append(word)
    test_talk_word.append(temp)
    
print(len(test_talk_word[0]))

208


In [45]:
test_talk_word[0][:10]

['fire',
 'nine',
 'days',
 'archive',
 '175',
 'films',
 '16',
 'millimeter',
 'negative',
 'books']

In [47]:
train_sent = []

for talk in train_data:
    for sent in talk:
        train_sent.append(sent)

In [40]:
from gensim.models import Word2Vec

In [49]:
model_ted = Word2Vec(train_sent, min_count=1, size=100)

In [53]:
train_talk_word_emb = []

for talk in train_talk_word:
    temp = []
    for word in talk:
        if word in model_ted.vocab.keys():
            temp.append(model_ted[word])
        else:
            temp.append(np.zeros(100))
    train_talk_word_emb.append(temp)

In [54]:
test_talk_word_emb = []

for talk in test_talk_word:
    temp = []
    for word in talk:
        if word in model_ted.vocab.keys():
            temp.append(model_ted[word])
        else:
            temp.append(np.zeros(100))
    test_talk_word_emb.append(temp)

In [66]:
train_lenghts[:10]

[475, 807, 852, 493, 971, 823, 676, 692, 977, 622]

In [87]:
train_talk_word_emb_exp = []

for i in range(len(train_talk_word_emb)):
    temp = []
    for j in range(max(train_lenghts)):
        if j <= (train_lenghts[i]-1):
            temp.append(train_talk_word_emb[i][j])
        else:
            temp.append(np.zeros(100))
    train_talk_word_emb_exp.append(temp)

In [85]:
test_talk_word_emb_exp = []

for i in range(len(test_talk_word_emb)):
    temp = []
    for j in range(max(train_lenghts)):
        if j <= (test_lenghts[i]-1):
            temp.append(test_talk_word_emb[i][j])
        else:
            temp.append(np.zeros(100))
    test_talk_word_emb_exp.append(temp)

In [86]:
np.shape(test_talk_word_emb_exp)

(250, 2221, 100)

In [88]:
np.shape(train_talk_word_emb_exp)

(1835, 2221, 100)

In [89]:
train_lenghts = np.array(train_lenghts)
train_talk_word_emb_exp = np.array(train_talk_word_emb_exp)

tf.reset_default_graph()

cell = tf.nn.rnn_cell.LSTMCell(num_units=100, state_is_tuple=True)

outputs, last_states = tf.nn.dynamic_rnn(
    cell=cell,
    dtype=tf.float64,
    sequence_length=train_lenghts[:1],
    inputs=train_talk_word_emb_exp[:1])

result = tf.contrib.learn.run_n(
    {"outputs": outputs, "last_states": last_states},
    n=1,
    feed_dict=None)

#assert result[0]["outputs"].shape == (1, 100, 100)
print(result[0]["outputs"])

# Outputs for the second example past past length 6 should be 0
#assert (result[0]["outputs"][1,7,:] == np.zeros(cell.output_size)).all()

[[[ 0.00335423  0.00456646  0.11484939 ..., -0.09846282 -0.01592748
   -0.04922342]
  [ 0.07969373  0.02491246  0.19337345 ...,  0.10140288  0.01015642
    0.02996071]
  [-0.01767184  0.01839419  0.22493143 ...,  0.03654385  0.01477763
   -0.0328414 ]
  ..., 
  [ 0.          0.          0.         ...,  0.          0.          0.        ]
  [ 0.          0.          0.         ...,  0.          0.          0.        ]
  [ 0.          0.          0.         ...,  0.          0.          0.        ]]]


In [69]:
tf.reset_default_graph()

# Create input data
X = np.random.randn(2, 10, 8)

# The second example is of length 6 
X[1,6:] = 0
X_lengths = [10, 6]

cell = tf.nn.rnn_cell.LSTMCell(num_units=64, state_is_tuple=True)

outputs, last_states = tf.nn.dynamic_rnn(
    cell=cell,
    dtype=tf.float64,
    sequence_length=X_lengths,
    inputs=X)

result = tf.contrib.learn.run_n(
    {"outputs": outputs, "last_states": last_states},
    n=1,
    feed_dict=None)

assert result[0]["outputs"].shape == (2, 10, 64)
print(result[0]["outputs"])

# Outputs for the second example past past length 6 should be 0
assert (result[0]["outputs"][1,7,:] == np.zeros(cell.output_size)).all()

[[[ 0.08330847 -0.07997911 -0.15619285 ...,  0.02258903  0.01590532
   -0.1110686 ]
  [ 0.03133713 -0.11231508 -0.10753537 ...,  0.09328638 -0.06160749
    0.12643219]
  [ 0.02809332 -0.15213351 -0.14210787 ..., -0.00432295  0.01988201
    0.02255423]
  ..., 
  [ 0.12359908  0.06268023 -0.07288837 ...,  0.10977463 -0.11880172
    0.17737559]
  [ 0.0284743   0.10941476 -0.06975772 ...,  0.14912158 -0.12484405
    0.20878322]
  [ 0.07652177  0.12213089 -0.09808181 ...,  0.18370856 -0.06454782
    0.06134945]]

 [[-0.06294364 -0.09950342 -0.10851849 ...,  0.06565711 -0.07177848
   -0.05096928]
  [-0.23090904 -0.02724335  0.0833625  ...,  0.11698978 -0.01457705
    0.0556754 ]
  [-0.15835107 -0.09366613  0.10595192 ..., -0.01740723 -0.09518339
    0.02513047]
  ..., 
  [ 0.          0.          0.         ...,  0.          0.          0.        ]
  [ 0.          0.          0.         ...,  0.          0.          0.        ]
  [ 0.          0.          0.         ...,  0.          0.     

In [106]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.05)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [117]:
sess.run(accuracy, feed_dict={x: test_bow_original, y_: test_labels_onehot})

0.292

In [62]:
X = np.random.randn(2, 10, 8)

In [63]:
X

array([[[ 0.14678782, -0.9460397 , -1.2041714 , -0.1513634 ,  0.28513941,
         -0.47494546, -1.79833339,  2.9269409 ],
        [-0.13164233, -0.66136219, -0.97029053,  0.53860253, -1.8829656 ,
          2.47401193, -0.36802385, -0.39040525],
        [-2.54057726,  0.54829754, -0.32072238, -0.71196374, -0.313327  ,
         -1.7544326 ,  0.33193185,  0.75610857],
        [-0.18969196,  0.98213862,  0.05066111, -0.96430621,  1.51850446,
          1.38648442, -0.94854702,  0.62405132],
        [-0.98470408,  0.14037008, -0.16054359,  0.92932763, -1.36672556,
          0.61382861, -0.09103976,  1.19416014],
        [-0.11254063, -0.98045208, -1.43099371, -0.39172266,  0.16338319,
          1.16809349,  0.8677304 ,  0.04259929],
        [-2.81111724,  0.4999473 , -1.08116719,  1.09862161, -1.15929163,
          1.01611669, -0.2044575 ,  0.26523627],
        [-0.4517384 ,  0.11223707, -1.6342156 , -0.58377012, -0.50358152,
          0.24490418, -0.69614575,  0.87229331],
        [ 0.6260